In [155]:
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [156]:
df = pd.read_csv("data_tambahan_35_baris_perkelas.csv") 

In [157]:


# Label encode untuk kolom Ruangan
le_label = LabelEncoder()
df['label_encoded'] = le_label.fit_transform(df['Ruangan'])

# Label encode untuk kolom mac
le_mac = LabelEncoder()
df['mac_encoded'] = le_mac.fit_transform(df['mac'])


In [158]:
X = df[['mac_encoded', 'RSSI', 'Lantai']]
y = df['label_encoded']

In [159]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

In [160]:
models = {
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "KNN": KNeighborsClassifier(),
    "Logistic Regression": LogisticRegression(max_iter=2000),
    "SVM": SVC()
}

In [161]:
best_model = None
best_model_name = ""
best_score = 0


# Evaluasi semua model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"\n=== {name} ===")
    print(f"Akurasi: {acc:.2f}")
    print(classification_report(
    y_test, y_pred,
    labels=np.unique(y_test),
    target_names=le_label.inverse_transform(np.unique(y_test)),
    zero_division=0
    
))
    if acc > best_score:
        best_score = acc
        best_model = model
        best_model_name = name
    print(f"Model terbaik : {best_model_name} dengan akurasi {best_score:.2f}")



=== Random Forest ===
Akurasi: 0.96
                                       precision    recall  f1-score   support

          Depan satuan pengawas (LT3)       0.50      0.43      0.46         7
                   Kepala lpmpp (LT3)       1.00      1.00      1.00         7
                    Kepala lppm (LT3)       1.00      1.00      1.00         7
                L2 Depan Tangga (LT2)       1.00      1.00      1.00         7
                     Lorong BMN (LT2)       1.00      1.00      1.00         7
               Ruang kabag umum (LT2)       1.00      1.00      1.00         7
                Sekretaris lppm (LT3)       1.00      1.00      1.00         8
   Toilet 1 samping kepala biro (LT2)       1.00      1.00      1.00         8
               area pintu depan (LT2)       1.00      1.00      1.00         7
         area tunggu depan lift (LT1)       1.00      1.00      1.00         7
                   depan lift 2 (LT2)       0.89      1.00      0.94         8
           kam

c:\Users\toffa\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 2000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=2000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [162]:
# Prediksi Data Baru

mac_asli = "06:0D:9E:25:7B:19"  # ganti sesuai MAC yang ingin diuji
rssi = -70
lantai = 1

# Encode MAC address dengan encoder yang sudah dilatih
mac_encoded = le_mac.transform([mac_asli])[0]

# Buat DataFrame data baru
data_baru = pd.DataFrame({
    'mac_encoded': [mac_encoded],
    'RSSI': [rssi],
    'Lantai': [lantai]
})

# Lakukan prediksi
prediksi_label = best_model.predict(data_baru)

# Konversi label angka ke nama ruangan
prediksi_ruangan = le_label.inverse_transform(prediksi_label)

print("\n=== Prediksi Data Baru ===")
print(f"MAC : {mac_asli}")
print(f"RSSI : {rssi}")
print(f"Prediksi Ruangan: {prediksi_ruangan[0]}")



=== Prediksi Data Baru ===
MAC : 06:0D:9E:25:7B:19
RSSI : -70
Prediksi Ruangan: lorong perencanaan (LT1)
